In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [71]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import datetime as dt
from pathlib import Path
from data_functions import  create_causal_data, get_treated_controls, control_outcomes, make_MYR_to_date
from multi_outcomes import multi_outcomes_data


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [4]:
%%time
canary_p = '../../temp/CANARY.p'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pd.read_pickle(canary_p)
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)

CPU times: user 27.9 s, sys: 9.41 s, total: 37.3 s
Wall time: 19.4 s


In [5]:
%%time
pd.DataFrame(biometrics.groupby(by = 'BIOMETRIC_CATEGORY').apply(len)).T

CPU times: user 19.7 s, sys: 5.02 s, total: 24.8 s
Wall time: 24.7 s


BIOMETRIC_CATEGORY     A1C       BMI    BNP  Blood Pressure (Diastolic)  Blood Pressure (Systolic)  Cholesterol (LDL)  DCSI-EGFR  DCSI-MICROALBUMIN  DCSI-SERUM  Ferritin    Height  Hematocrit  Hemoglobin    Iron  NT_proBNP     PSA  Potassium  Troponin    Weight    eGFR
0                   790853  10034183  42552                     4902004                    5238991            1305880     628758              81297     2734084    219613  11808045     2574412     2919909  197878       9258  310283    2666981     94030  14000267  907656

In [6]:
%%time
merged, treated,  controls, canary2 = get_treated_controls(canary.copy(), pmpm.copy(), biometrics.copy(), 
                                                          merge_how = 'left')

CPU times: user 1min 12s, sys: 23 s, total: 1min 35s
Wall time: 1min 35s


In [15]:
len(merged), len(treated)+len(controls)

(7286398, 7286398)

In [64]:
#copy controls to use if necessary
controls2 = controls.copy()

In [65]:
max_days = 180
min_days = 120
threshold = 27
window = 3

In [66]:
controls2[controls2.RESULT_VALUE.isnull()].shape

(6105406, 78)

# 1 sort by Id and MYR and front fill

In [364]:
%%time
controls2 = controls2.sort_values(by = ['PERS_ID', 'MYR']).reset_index(drop = True)
controls2.RESULT_VALUE = controls2.groupby('PERS_ID').RESULT_VALUE.fillna(method = 'ffill', limit = window)

CPU times: user 1min 3s, sys: 3.66 s, total: 1min 7s
Wall time: 1min 6s


In [365]:
controls2[controls2.RESULT_VALUE.isnull()].shape

(3409961, 79)

# 2 set Result value 2 based on thresholding and bigger better

In [366]:
controls2['RESULT_VALUE2'] = (controls2.RESULT_VALUE <= threshold).astype(int)

# 3 whatever is Nan in Result value put it as 1 in Result value 2

In [367]:
controls2.loc[controls2.RESULT_VALUE.isnull(), 'RESULT_VALUE2'] = 1

# 4 copy MYR to event date and make it a datetime date object

In [368]:
controls2.EVENT_DATE = controls2.MYR.apply(make_MYR_to_date)

In [463]:
%%cython
#cimport numpy 
cimport cython
from cython.parallel import prange

import numpy as np
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True)

def  make_cy(double[:,:] outcomes, double[:,:] values, long[:] days, int min_days, int max_days, int size):
    
    cdef int i = 0
    cdef int j
    cdef long day
    cdef long day_diff
    cdef int flag
    cdef double val, val2, chosen_val, chosen_val2
    cdef double o1, o2, o3
    cdef double NAN = np.nan
    for i in range(size):
        day = days[i]
        val = values[i,0]
        val2 = values[i,1]
        flag = 0
        
        for j in range(i+1, size):
            day_diff = days[j] - day
           # print(day_diff)
            if (day_diff >= min_days) & (day_diff <= max_days):
                flag = 1
                chosen_val = values[j,0]
                chosen_val2 = values[j,1]

                if (chosen_val - val) <0:
                    o1 = 1.0
                else:
                    o1 = 0.0
                    
                o2 = (chosen_val - val)/val
                o3 = chosen_val2-val2
                
                outcomes[i, 0 ] = o1
                outcomes[i,1] = o2
                outcomes[i, 2] = o3
                break
        
        if flag == 0:
            outcomes[i, 0 ] = NAN
            outcomes[i,1] = NAN
            outcomes[i, 2] = NAN
    
    
            
                
                
            
            
        

# 5 Create the Outcomes 1, 2, 3

In [464]:
def datetime_to_days(group):
    
    days = group.year*365 + group.month*30 + group.day
    return days


def make_control_outcomes_cy(group, min_days, max_days):
    """
    make control outcomes
    
    """
    
    days = group.DAYS.values
    outcomes = np.zeros(shape = [len(group),3], dtype = float)
    result_values = group[['RESULT_VALUE', 'RESULT_VALUE2']].values
    size = int(len(group))
    
    make_cy(outcomes, result_values, days, int(min_days), int(max_days),  int(size))
   # print(outcomes)
    group[['OUTCOME1', 'OUTCOME2', 'OUTCOME3']] = np.array(outcomes)
    
    return group.reset_index(drop = True)

    
    

def make_control_outcomes(group, min_days, max_days):
    """
    group needs to be sorted by MYR
    
    """
    #group['OUTCOME1'] = 0
    #group['OUTCOME2'] = 0
    #group['OUTCOME3'] = 0
    dates = group.EVENT_DATE
    group = group.reset_index(drop = True)
    
  
   # to_keep = []
    outcomes = []
    result_values = group[['RESULT_VALUE', 'RESULT_VALUE2']].values
    for _,i in enumerate(group.index):
       # print(i)
        datei = group.loc[i, 'EVENT_DATE']
        difference = (dates - datei).dt.days
        filteri = (difference >= min_days) & (difference <= max_days)
        if  filteri.any():
            
           # to_keep.append(i)
            #gg = group[filteri]
            #chosen_val = gg.iloc[0].RESULT_VALUE
            #chosen_val2 = gg.iloc[0].RESULT_VALUE2
           # val = group.loc[i,'RESULT_VALUE']
           # val2 = group.loc[i,'RESULT_VALUE2']
           # group.loc[i, 'OUTCOME1'] = int((chosen_val - val) < 0)
           # group.loc[i, 'OUTCOME2'] = (chosen_val - val)/val
           # group.loc[i, 'OUTCOME3'] = int(chosen_val2-val2)
            
            gg = result_values[filteri]
            chosen_val = gg[0,0]
            chosen_val2 = gg[0,1]

            val = result_values[i,0]
            val2 = result_values[i,1]
            o1 = int((chosen_val - val) < 0)
            o2 = (chosen_val - val)/val
            o3 = int(chosen_val2-val2)
            outcomes.append([o1,o2,o3])
            #outcomes[i,0] = o1
            #outcomes[i,1] = o2
            #outcomes[i,2] = o3
        else:
            #group.loc[i, 'OUTCOME1'] = np.nan
            #group.loc[i, 'OUTCOME2'] = np.nan
            #group.loc[i, 'OUTCOME3'] = np.nan
            outcomes.append([np.nan, np.nan, np.nan])
            #outcomes[i] = np.nan
            
    group[['OUTCOME1', 'OUTCOME2', 'OUTCOME3']] = np.array(outcomes)
    return group.reset_index(drop = True)

    
    
    
    

In [465]:
ids = controls2.PERS_ID.unique()
con = controls2[controls2.PERS_ID.isin(ids[0:100])].reset_index(drop = True)
con['OUTCOME1'] = 0
con['OUTCOME2'] = 0
con['OUTCOME3'] = 0
con['DAYS'] = con.EVENT_DATE.apply(datetime_to_days)

In [466]:
%%time

cont_test3 = con.groupby('PERS_ID').apply(make_control_outcomes_cy, min_days, max_days).reset_index(drop = True).fillna(value = -1000)
#cont_test4 = con.groupby('PERS_ID').apply(make_control_outcomes, min_days, max_days).reset_index(drop = True).fillna(value = -1000)

CPU times: user 355 ms, sys: 47 µs, total: 355 ms
Wall time: 354 ms


In [467]:
%%time
cont_test4 = con.groupby('PERS_ID').apply(make_control_outcomes,
                                          min_days, max_days).reset_index(drop = True).fillna(value = -1000)

CPU times: user 3.98 s, sys: 0 ns, total: 3.98 s
Wall time: 3.98 s


In [414]:
cont_test3.shape

(34628, 83)

In [469]:
(cont_test3 == cont_test4).all()

PERS_ID                        True
AGE_AT_MIDMONTH                True
ALLOW_AMT                      True
CC_ADHD                        True
CC_ALZHEIMER                   True
CC_ALZHEIMER_PLUS              True
CC_AMI                         True
CC_ANEMIA                      True
CC_ANXIETY_DISORDER            True
CC_ARTHRITIS                   True
CC_ASTHMA                      True
CC_ATRIAL_FIB                  True
CC_AUTISM                      True
CC_BIPOLAR                     True
CC_BPH                         True
CC_BREAST_CANCER               True
CC_CATARACT                    True
CC_CEREBRAL_PALSY              True
CC_CHRONIC_KIDNEY              True
CC_COLORECTAL_CANCER           True
CC_COPD                        True
CC_CYSTIC_FIBROSIS             True
CC_DEPRESSION                  True
CC_DEPRESSIVE_DISORDERS        True
CC_DEVELOP_DELAYS              True
CC_DIABETES                    True
CC_DISAB_DX_CNT                True
CC_DX_CNT                   

In [474]:
cont_test4[cont_test3.OUTCOME3 != cont_test4.OUTCOME3].OUTCOME3

1135   -1000.0
1955   -1000.0
2169   -1000.0
2702   -1000.0
3159   -1000.0
Name: OUTCOME3, dtype: float64

In [483]:
cont_test4[['PERS_ID','EVENT_DATE','DAYS', 'OUTCOME3']].loc[1129: 1139]

PERS_ID  EVENT_DATE    DAYS  OUTCOME3
1129  2565637  2018-08-01  736811       0.0
1130  2565637  2018-09-01  736841       0.0
1131  2565637  2018-10-01  736871       0.0
1132  2565637  2018-11-01  736901       0.0
1133  2565637  2018-12-01  736931       0.0
1134  2565637  2019-01-01  736966       0.0
1135  2565637  2019-02-01  736996   -1000.0
1136  2565637  2019-03-01  737026       0.0
1137  2565637  2019-04-01  737056       0.0
1138  2565637  2019-05-01  737086   -1000.0
1139  2565637  2019-08-01  737176   -1000.0

In [415]:
cont_test3.dropna(subset = ['OUTCOME3'], inplace = True)

cont_test3.drop(columns = ['DAYS'], inplace = True)


cont_test3